# КП 1. Диференціальний криптоаналіз блокових шифрів.

Виконали студенти групи ФІ-32мн Костянтин Баєвський та Денис Шифрін. Варіант обрали 16, за номером Шифріна в списку групи.

## Мета роботи

Опанування сучасних методів криптоаналізу блокових шифрів, набуття навичок у дослідженні стійкості блокових шифрів до диференціального криптоаналізу.

## Опис труднощів, що виникали, та шляхів їх розв’язання

1. Алгоритм пошуку високоімовірних диференціалів та згодом пошук ключів також не були тривіальним, тому знадобився час на коректне написання.
2. Ми звикли писати мовами програмування із строгою типізацією, Java та C#, але виникло багато проблем в роботі із файлами та бітами, тому було прийнято рішення перейти на python для виконання даного практикуму.
3. Нам досі не до кінця зрозумілі принципи роботи з потоками та процесами в python, але після купи гайдів ми таки змогли здобути достатньо знань для написання практикуму.
4. Взаємодія із представленим застосунком "heys.exe". Потрібно було викликати програму мовою програмування, а потім симулювати натискання клавіші ентер, щоб закрити пул. 
5. Час очікування результатів роботи на наших обчислювальних машинах був досить великим, через слабке залізо.

## Етап підготовки

### Необхідні імпорти

In [1]:
from collections import Counter
from multiprocess import Pool
from random import randrange
import subprocess
import numpy as np
import pandas as pd

## Завдання 1. 

Реалізувати шестираундовий шифр Хейса. Ключем шифрування вважати довільний 112-бітний бітовий рядок, який розбивається на сім незалежних 16-бітних раундових підключа.

In [2]:
class Heys:
    def __init__(self):
        self.s_block = [0x4, 0x3, 0xE, 0xD, 0x5, 0x0, 0x2, 0xB, 0x1, 0xA, 0x7, 0x6, 0x9, 0xF, 0x8, 0xC]
        self.s_block_inverse = [0x5, 0x8, 0x6, 0x1, 0x0, 0x4, 0xB, 0xA, 0xE, 0xC, 0x9, 0x7, 0xF, 0x3, 0x2, 0xD]

    def round(self, block: int, round_key: int=0x0):
        y = block ^ round_key
        y_s = self.substitute(y)
        y_p = self.permute(y_s)
        return y_p

    def round_(self, block: int, round_key: int=0x0) -> int:
        y_k = block ^ round_key
        y_p = self.permute(y_k)
        y_s = self.substitute_inverse(y_p)
        return y_s

    def substitute(self, block: int):
        return (self.s_block[(block >> 12) & 0xF] << 12) | (self.s_block[(block >> 8) & 0xF] << 8) |\
             (self.s_block[(block >> 4) & 0xF] << 4) | (self.s_block[block & 0xF])
    
    def substitute_inverse(self, block: int):
        return (self.s_block_inverse[(block >> 12) & 0xF] << 12) | (self.s_block_inverse[(block >> 8) & 0xF] << 8) |\
             (self.s_block_inverse[(block >> 4) & 0xF] << 4) | (self.s_block_inverse[block & 0xF])

    def permute(self, block: int):
        result = 0
        for i in range(4):
            for j in range(4):
                result |= (block >> (i * 4 + j) & 1) << (j * 4 + i) 
        return result


## Завдання 2. 

Реалізувати пошук високоімовірних п’ятираундових диференціалів шифру Хейса методом "гілок та границь".

### Таблиця диференціальних імовірностей

In [3]:
def get_table():
    heys = Heys()
    diff_substitution = np.zeros((16, 16))

    for alpha in range(16):
        for beta in range(16):
            for x in range(16):
                diff_substitution[alpha][beta] += heys.substitute(x ^ alpha) == heys.substitute(x) ^ beta

    diff_substitution /= 16
    return diff_substitution

pd.DataFrame(get_table())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.0,0.125,0.000,0.125,0.125,0.125,0.125,0.125,0.000,0.125,0.000,0.125,0.000,0.000,0.000,0.000
2,0.0,0.125,0.000,0.125,0.000,0.000,0.125,0.125,0.000,0.000,0.125,0.125,0.125,0.000,0.125,0.000
3,0.0,0.000,0.125,0.000,0.000,0.125,0.000,0.250,0.000,0.125,0.000,0.000,0.000,0.250,0.125,0.000
4,0.0,0.125,0.000,0.125,0.000,0.125,0.125,0.000,0.125,0.000,0.125,0.000,0.125,0.000,0.000,0.125
5,0.0,0.000,0.000,0.125,0.125,0.125,0.125,0.000,0.000,0.000,0.000,0.125,0.000,0.000,0.250,0.125
6,0.0,0.000,0.000,0.000,0.000,0.000,0.250,0.000,0.125,0.250,0.000,0.125,0.000,0.125,0.125,0.000
7,0.0,0.125,0.125,0.000,0.000,0.000,0.000,0.000,0.250,0.000,0.000,0.000,0.000,0.125,0.125,0.250
8,0.0,0.000,0.000,0.000,0.000,0.125,0.000,0.125,0.000,0.250,0.125,0.125,0.125,0.000,0.000,0.125
9,0.0,0.000,0.125,0.125,0.000,0.000,0.000,0.000,0.125,0.125,0.250,0.000,0.000,0.000,0.250,0.000


### 2.1. Алгоритм пошуку високоімовірних диференціалів ітеративних шифрів

$$
\begin{array}{l}
1. \ \Gamma_0 (\alpha) := \{ (\alpha, 1) \}. \\
2. \ \text{Для всіх } t \text{ від } 1 \text{ до } r \text{ виконати:} \\
    \quad 2.1 \ \Gamma_t (\alpha) := \varnothing. \\
    \quad 2.2 \ \text{Для кожної пари } (\beta_i, p_i) \in \Gamma_{t-1} (\alpha) \text{ виконати:} \\
    \quad \quad \text{// побудова «гілок»} \\
    \quad \quad 2.2.1 \ \text{Для кожної пари } (\gamma_j, q_j) \in \Delta (\beta_i) \text{ виконати:} \\
        \quad \quad \quad 2.2.1.1 \ \text{Якщо } (\gamma_j, p(\gamma_j)) \in \Gamma_t (\alpha), \text{ то:} \\
            \quad \quad \quad \quad \text{а.} \ p(\gamma_j) = p(\gamma_j) + p_i \cdot q_j; \\
        \quad \quad \quad 2.2.1.2 \ \text{інакше:} \\
            \quad \quad \quad \quad \text{а.} \ \text{включити у } \Gamma_t (\alpha) \text{ пару } (\gamma_j, p_i \cdot q_j). \\
    \quad 2.3 \ \text{Для кожної пари } (\gamma_i, p_i) \in \Gamma_t (\alpha) \text{ виконати:} \\
    \quad \quad \text{// перевірка «границь»} \\
        \quad \quad 2.3.1 \ \text{Якщо } p_i \leq p^*, \text{ то:} \\
            \quad \quad \quad \text{а.} \ \text{вилучити пару } (\gamma_i, p_i) \text{ з } \Gamma_t (\alpha). \\
\end{array}
$$



Сенс алгоритму полягає в тому, що основна задача гілкується, тобто розбивається на менші підзадачі, та обмежується границями. В залежності від значень границь задача може бути відкинута, чи розбита знов. Процес відбувається доти, доки існують менші підзадачі.

Для оптимізації також можна використати передобчислені диференціали в минулому пункті.

In [4]:
def get_differences(beta: int):
    betas = []
    cipher = Heys()
    for text in range(2**16):
        betas.append(cipher.round(text) ^ cipher.round(text ^ beta))

    return Counter(betas)

In [5]:
def diffSearch(alpha: int=1, P: list[float]=[0.1] * 6, r: int = 6):
    
    # Пункт 2.1.
    L = [[]] * r
    
    # Пункт 1.
    L[0] = [tuple([alpha, 1.0])]
    
    # Пункт 2.
    for t in range(1,r):
        
        # Пункт 2.2.
        for (beta, p) in L[t - 1]:
            
            # Пункт 2.2.1.
            gammas = sorted([tuple([difference[0], difference[1] / 2**16]) for difference in get_differences(beta).items()], key=lambda difference: difference[1], reverse=True)
            
            for (gamma, q) in gammas:
                
                L_t = L[t].copy()
                if gamma in [l[0] for l in L_t]:
                    
                    # Пункт 2.2.1.1.
                    pg = list(filter(lambda x : x[0] == gamma , L_t))[0][1]
                    L_t.remove((gamma, pg))
                    L_t.append((gamma, pg + (p * q)))
                else: 
                    # Пункт 2.2.1.2.
                    L_t.append(tuple([gamma, p * q]))
                
                L[t] = L_t.copy()
        
        L_t = L[t].copy()
        # Пункт 2.3.
        for (gamma, p) in L[t]:
            if p <= P[t]: # Пункт 2.3.1.
                L_t.remove((gamma, p))
        L[t] = L_t.copy()
    return L
    

In [6]:
def find_candidates():
    candidates = {}
    for i in range(4):
        for j in range(1, 16):
            _alpha = j << (4 * i)
            candidates[_alpha] = diffSearch(alpha=_alpha)
    return candidates

In [7]:
possible_candidates = find_candidates()

In [8]:
possible_candidates

{1: [[(1, 1.0)],
  [(273, 0.125),
   (17, 0.125),
   (257, 0.125),
   (4097, 0.125),
   (4113, 0.125),
   (1, 0.125),
   (272, 0.125),
   (256, 0.125)],
  [],
  [],
  [],
  []],
 2: [[(2, 1.0)],
  [(4112, 0.125),
   (4368, 0.125),
   (273, 0.125),
   (4113, 0.125),
   (272, 0.125),
   (4352, 0.125),
   (1, 0.125),
   (17, 0.125)],
  [],
  [],
  [],
  []],
 3: [[(3, 1.0)],
  [(4353, 0.25),
   (273, 0.25),
   (4097, 0.125),
   (4368, 0.125),
   (16, 0.125),
   (257, 0.125)],
  [],
  [],
  [],
  []],
 4: [[(4, 1.0)],
  [(1, 0.125),
   (17, 0.125),
   (4352, 0.125),
   (272, 0.125),
   (4096, 0.125),
   (257, 0.125),
   (4369, 0.125),
   (4112, 0.125)],
  [],
  [],
  [],
  []],
 5: [[(5, 1.0)],
  [(4368, 0.25),
   (256, 0.125),
   (272, 0.125),
   (257, 0.125),
   (4369, 0.125),
   (17, 0.125),
   (4113, 0.125)],
  [],
  [],
  [],
  []],
 6: [[(6, 1.0)],
  [(272, 0.25),
   (4097, 0.25),
   (4096, 0.125),
   (4113, 0.125),
   (4353, 0.125),
   (4368, 0.125)],
  [],
  [],
  [],
  []],
 7: [[

## Завдання 3. 

Реалізувати атаку на сьомий раундовий ключ шифру Хейса.

Для реалізації атаки необхідно:
   1. Накопичити статистичний матеріал. Накопичуємо пари відкритих текстів $(X, X_{mut})$, де $X_{mut} = X \oplus \alpha$. Шифруємо їх та отримуємо тексти $(С, С_{mut})$ відповідно.
   2. Обчислити кількість успішних випробувань. Для кожного гіпотетичного ключа $k_r$ треба розшифрувати пари $(С, С_{mut})$ та отримати пари значень $(Y, Y_{mut})$. Далі перевіряємо гіпотезу $Y \oplus Y_{mut} = \beta$. Зауважимо, що кандидати та їх $(\alpha, \beta)$ заздалегідь відомо. 
   3. Фільтруємо кандидатів. Вибираємо кандидатів із максимальною кількістю успішних випробувань.

### 1. Накопичити статистичний матеріал.

In [9]:
def get_candidates():
    result = []
    possible_candidates = find_candidates()

    for key in possible_candidates.keys():
        for (beta, probability) in (pair for sublist in list(possible_candidates[key]) for pair in sublist):
            result.append((key, beta))
    print(result)
    return result

In [10]:
def read_file(file):
    with open(file, 'rb') as f:
        data = f.read()
    return [(data[i+1] << 8) | data[i] for i in range(0, len(data), 2)]

In [11]:
def write_file(file, text: list[int]):
    with open(file, 'wb') as f:
        f.write(bytearray([byte for x in text for byte in x.to_bytes(2, byteorder='little')]))

In [12]:
def get_ciphertext(filefrom, fileto):
    subprocess.Popen(fr'.\heys.exe e 16 {filefrom} {fileto}', shell=True, stdout=subprocess.DEVNULL, stderr=None).communicate(input="\x0a".encode())

    return read_file(fileto)

In [13]:
def generate_ciphertext(alpha, samples_num, no):
    plaintext = [randrange(0, 2**16) for _ in range(0, samples_num)]

    write_file(fr'./files/plain_{no}.bin', plaintext)
    write_file(fr'./files/plain_{no}_m.bin', [text ^ alpha for text in plaintext])
    
    write_file(fr'./files/cipher_{no}.bin', "")
    write_file(fr'./files/cipher_{no}_m.bin', "")
    
    return get_ciphertext(fr'./files/plain_{no}.bin', fr'./files/cipher_{no}.bin'), get_ciphertext(fr'./files/plain_{no}_m.bin', fr'./files/cipher_{no}_m.bin')

### 3. Фільтруємо кандидатів.

In [14]:
def check_candidate(cipher, cipher_m, k, beta):
    success_rate = 0
    heys_obj = Heys()
    for (ct1, ct2) in zip(cipher, cipher_m):
        pt1, pt2 = heys_obj.round(ct1, k), heys_obj.round(ct2, k)
        if pt1 ^ pt2 == beta:
            success_rate += 1
    return success_rate

### 2. Атака

In [15]:
def Attack():
    # checking keys from 0..=((1<<16)-1)
    keys = [(i, 0) for i in range(0, 2**16)]

    candidates = get_candidates()

    for i, (alpha, beta) in enumerate(candidates):
        ct_nonmut, ct_mut = generate_ciphertext(alpha, 7000, i)

        # 2. Обчислити кількість успішних випробувань. 
        with Pool() as pool:
            success_rates = list(pool.map(lambda k: check_candidate(ct_nonmut, ct_mut, k[0], beta), keys))

        max_rate = max(success_rates)
        key_list_with_max_rate = []
        for j, (k, _) in enumerate(keys):
            if success_rates[j] == max_rate:
                key_list_with_max_rate.append((k, success_rates[j]))
        keys = key_list_with_max_rate

        print("\n\n Iteration: {0}, keys_hex: {1}, keys: {2}, max key rate: {3}\n\n".format(i,
                                                                                            '[{}]'.format(', '.join((hex(x[0])) for x in keys)),
                                                                                            keys,
                                                                                            max_rate))
        
        if len(keys) == 1:
            break

In [16]:
%%time
Attack()

[(1, 1), (1, 273), (1, 17), (1, 257), (1, 4097), (1, 4113), (1, 1), (1, 272), (1, 256), (2, 2), (2, 4112), (2, 4368), (2, 273), (2, 4113), (2, 272), (2, 4352), (2, 1), (2, 17), (3, 3), (3, 4353), (3, 273), (3, 4097), (3, 4368), (3, 16), (3, 257), (4, 4), (4, 1), (4, 17), (4, 4352), (4, 272), (4, 4096), (4, 257), (4, 4369), (4, 4112), (5, 5), (5, 4368), (5, 256), (5, 272), (5, 257), (5, 4369), (5, 17), (5, 4113), (6, 6), (6, 272), (6, 4097), (6, 4096), (6, 4113), (6, 4353), (6, 4368), (7, 7), (7, 4369), (7, 4096), (7, 1), (7, 4368), (7, 4353), (7, 16), (8, 8), (8, 4097), (8, 257), (8, 4113), (8, 4352), (8, 4369), (8, 4112), (8, 273), (9, 9), (9, 4368), (9, 4112), (9, 16), (9, 4096), (9, 4097), (9, 17), (10, 10), (10, 17), (10, 257), (10, 4369), (10, 273), (10, 4353), (10, 4352), (10, 4113), (10, 256), (11, 11), (11, 16), (11, 256), (11, 4352), (11, 4097), (11, 4096), (11, 4353), (12, 12), (12, 4353), (12, 4352), (12, 272), (12, 1), (12, 256), (12, 4112), (12, 257), (13, 13), (13, 4113),

## Висновки

В ході виконання практикуму ми отримали шуканий ключ та на практиці перевірили стійкість шифру Хейса до атак, основаних на диференціальному криптоаналізі.